## Atelier 6 - Calculateur de variogramme théorique (multi-composantes anisotropes)

**But pédagogique** : Calcul direct du variogramme pour tout type de structure.

### Fonctionnalités :

- **Entrée** : jusqu’à 3 composantes :
  - Type de modèle : sphérique, exponentiel, gaussien, pépite
  - Variance, portée, anisotropie ($a_x$, $a_y$, angle) de chaque modèle
  - Distance ($h_x$ et $h_y$)

---
> 💡 **Note** : Pour le modèle de pépite, entrée n'importe qu'elle valeur pour les paramètres du modèle.
> 
> 💡 **Note** : Les distances peuvent être négatives.

In [3]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

def cov_component(h, range_major, range_minor, angle_deg, model, sill):
    angle_rad = np.deg2rad(angle_deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    hx, hy = h[0], h[1]
    h_rot = cos_a * hx + sin_a * hy
    v_rot = -sin_a * hx + cos_a * hy
    h_scaled = h_rot / range_major
    v_scaled = v_rot / range_minor
    dist = np.sqrt(h_scaled**2 + v_scaled**2)
    if model == 'Sphérique':
        if dist < 1:
            cov = sill * (1 - 1.5*dist + 0.5*dist**3)
        else:
            cov = 0
    elif model == 'Exponentiel':
        cov = sill * np.exp(-3 * dist)
    elif model == 'Gaussien':
        cov = sill * np.exp(-3 * dist**2)
    elif model == 'Nugget':
        cov = sill if dist == 0 else 0
    else:
        raise ValueError(f"Modèle inconnu : {model}")
    return cov

def covariance_multi(h, comps):
    cov_total = 0
    for c in comps:
        if c['active']:
            cov_total += cov_component(h, c['range_major'], c['range_minor'], c['angle_deg'], c['model'], c['sill'])
    return cov_total

def variogram(h, comps):
    sill_total = sum(c['sill'] for c in comps if c['active'])
    return sill_total - covariance_multi(h, comps)

def create_component_widget(index):
    active_cb = widgets.Checkbox(value=False, description=f"Comp. {index+1}")
    model_dd = widgets.Dropdown(
        options=['Sphérique', 'Exponentiel', 'Gaussien', 'Nugget'],
        value='Sphérique',
        description='Modèle:'
    )
    sill_txt = widgets.Text(value='1.0', description='Sill:')
    range_major_txt = widgets.Text(value='10.0', description='Portée maj.:')
    range_minor_txt = widgets.Text(value='10.0', description='Portée min.:')
    angle_txt = widgets.Text(value='0.0', description='Angle (°):')
    
    def toggle_inputs(change):
        enabled = change['new']
        model_dd.disabled = not enabled
        sill_txt.disabled = not enabled
        range_major_txt.disabled = not enabled
        range_minor_txt.disabled = not enabled
        angle_txt.disabled = not enabled
    
    active_cb.observe(toggle_inputs, names='value')
    toggle_inputs({'new': active_cb.value})
    
    comp_box = widgets.VBox([active_cb, model_dd, sill_txt, range_major_txt, range_minor_txt, angle_txt])
    return comp_box, active_cb, model_dd, sill_txt, range_major_txt, range_minor_txt, angle_txt

comp_widgets = []
for i in range(3):
    comp_widgets.append(create_component_widget(i))

hx_txt = widgets.Text(value='5.0', description='$h_x$:')
hy_txt = widgets.Text(value='0.0', description='$h_y$:')

btn_calc = widgets.Button(description="Calculer variogramme γ(h)")
output = widgets.Output()

def on_calc_clicked(b):
    with output:
        clear_output()
        try:
            comps = []
            for (comp_box, active_cb, model_dd, sill_txt, rmaj_txt, rmin_txt, angle_txt) in comp_widgets:
                active = active_cb.value
                if active:
                    model = model_dd.value
                    sill = float(sill_txt.value)
                    rmaj = float(rmaj_txt.value)
                    rmin = float(rmin_txt.value)
                    angle = float(angle_txt.value)
                else:
                    model = None
                    sill = 0
                    rmaj = 1
                    rmin = 1
                    angle = 0
                comps.append({
                    'active': active,
                    'model': model,
                    'sill': sill,
                    'range_major': rmaj,
                    'range_minor': rmin,
                    'angle_deg': angle
                })
            
            hx = float(hx_txt.value)
            hy = float(hy_txt.value)
            h = np.array([hx, hy])
            gamma = variogram(h, comps)
            print(f"Variogramme γ(h) pour h=({hx}, {hy}) = {gamma:.4f}")
        except Exception as e:
            print(f"Erreur : {e}")

btn_calc.on_click(on_calc_clicked)

ui = widgets.VBox([
    widgets.HTML("<h3>Calculateur de variogramme théorique (multi-composantes anisotropes)</h3>"),
    widgets.HBox([hx_txt, hy_txt]),
    widgets.HBox([w[0] for w in comp_widgets]),  # <-- composantes côte à côte
    btn_calc,
    output
])

display(ui)
